In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import json
import serial
import numpy as np
import time

In [ ]:
#File path for the dataset 
file_path ='./archive/dataset.csv'
data = pd.read_csv(file_path)

#create  label encoder
label_encoder = LabelEncoder()

#Apply label encoder to the "crop" column
data['Crop'] = label_encoder.fit_transform(data['Crop'])

print(data.head())
print(data.info())


In [6]:
#Features and labels
X = data.drop('Irrigation', axis = 1)
y= data[['Irrigation']]

#Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

#Ensure that y_train and y_test are 1D arrays
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

In [ ]:
# Train the model
model = RandomForestClassifier(n_estimators = 100, random_state = 42)
model.fit(X_train, y_train)

In [8]:
#Evaluate the model
y_pred = model.predict(X_test)
print(f'Accuracy:{accuracy_score(y_test, y_pred)}')


Accuracy:0.9207920792079208


In [9]:
#Save the model
joblib.dump(model, 'random_forest_model.pkl')

#Convert the label encoding mapping to the native Python types
crop_mapping = {str(key): int(value) for key, value in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))}

#Save the label encoding mapping
with open('crop_mapping.json', 'w') as f:
    json.dump(crop_mapping, f)

In [ ]:
#install sklearn_porter  to convert model to c language
!pip install sklearn-porter


In [ ]:
pip install m2cgen


In [22]:
import m2cgen as m2c

# Load your trained model
model = joblib.load('random_forest_model.pkl')

# Generate C code for the model
model_code = m2c.export_to_c(model)

# Save the model code to a file
with open('model.c', 'w') as f:
    f.write(model_code)
